In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
ls /kaggle/input/

Импорты необходимых библиотек

In [ ]:
!pip install torchtoolbox

In [ ]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.augmentations.dropout.cutout import Cutout
import torch
import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler
from sklearn.utils.class_weight import compute_class_weight
from os import listdir
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import time
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torchvision import models
import cv2
from torch.utils.data import Dataset
import pandas as pd
from torchtoolbox.tools import mixup_data, mixup_criterion

Конфиг

In [ ]:
# девайс
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# размер изображения
image_size = 256

# размер батча
batch_size = 64

# трансформации для обучения и валидации
transforms_train = A.Compose([
    A.Resize(image_size, image_size),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(30, p=0.3),
    A.RandomSizedCrop((image_size - 60, image_size), image_size, image_size, p=0.3),
    A.CoarseDropout(max_holes=5, max_height=50, max_width=50, min_holes=3, min_height=30, min_width=30),
    A.Normalize(),
    ToTensorV2()
])

transforms_val = A.Compose([
    A.Resize(image_size, image_size),
    A.Normalize(),
    ToTensorV2()
])

mixup = True
alpha=0.2

# используемая модель("resnet18", "resnet18_my")
model_name = "resnet18"
model = models.resnet18()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 256)
# model.load_state_dict(torch.load("./runs/run_train_9/weights/best.pt"))
model.to(device)

# количество эпох
num_epochs = 150

# лосс
label_smoothing = 0.1
# criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
# loss = "crossentropy"
loss = "crossentropybalance"
# loss = "kldiv"  # для использования этой штуки выход модели нужно обернуть в софтмакс

# оптимизатор
# optimizer = optim.AdamW(model.parameters(), lr=1e-2)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True, weight_decay=1e-4)

# шедулер(может быть None)  # https://www.kaggle.com/code/isbhargav/guide-to-pytorch-learning-rate-scheduling
# scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
# scheduler = None
# scheduler = lr_scheduler.CosineAnnealingLR(optimizer, 14, 1e-5)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
scheduler_name = "ReduceLROnPlateau"

In [ ]:
def train_model(
        model,
        criterion,
        optimizer,
        scheduler,
        dataloaders,
        dataset_sizes,
        num_epochs=25,
        save_path="./"
        ):
    since = time.time()
    path_save_best_model = save_path + "best.pt"
    best_acc = 0.0

    val_loss_lst = []
    train_loss_lst = []
    val_acc_lst = []
    lr_lst = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                if phase == "train" and mixup:
                    inputs, labels_a, labels_b, lam = mixup_data(inputs, labels, alpha)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)

                    if phase=="train" and mixup:
                        loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
                    else:
                        loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'train' and scheduler is not None:
                if scheduler_name == "ReduceLROnPlateau":
                    scheduler.step(epoch_loss)
                else:
                    scheduler.step()
                lr_lst.append(scheduler._last_lr[0])

            if phase == "train":
                train_loss_lst.append(epoch_loss)
            elif phase == "val":
                val_loss_lst.append(epoch_loss)
                val_acc_lst.append(epoch_acc.item())

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), path_save_best_model)

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    return {
        "train_lst": train_loss_lst,
        "val_lst": val_loss_lst,
        "val_acc_lst": val_acc_lst,
        "lr_lst": lr_lst
    }

In [ ]:
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

In [ ]:
class CaltechDataset(Dataset):
    def __init__(self, df, transforms):
        self.df = df
        self.transforms = transforms
        self.idx2label = {idx: self.df[self.df.label_idx == idx].label.values[0] for idx in set(self.df.label_idx.values)}
        self.label2idx = {label: idx for idx, label in self.idx2label.items()}
  
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx].path
        label_idx = self.df.iloc[idx].label_idx

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        aug = self.transforms(image=img)
        img = aug["image"]
    
        return img, int(label_idx)

    def __len__(self):
        return len(self.df)


def get_caltech_dataframe(curr_path: str) -> pd.DataFrame:
    with open(curr_path, "r") as f:
        data_rows = f.readlines()

    data_rows = [data_row[:-1] for data_row in data_rows]

    dict_data = {
        "path": [],
        "label_idx": [],
        "label": []
    }

    for item in data_rows:
        curr_path, curr_label_idx, curr_label = item.split(" ")
        dict_data["path"].append("/kaggle/input/caltech256/" + curr_path)
        dict_data["label_idx"].append(curr_label_idx)
        dict_data["label"].append(curr_label)

    df = pd.DataFrame(dict_data)

    return df

In [ ]:
df_train = get_caltech_dataframe("/kaggle/input/train-val-txt/train_lst.txt")
df_val = get_caltech_dataframe("/kaggle/input/train-val-txt/val_lst.txt")

# 4. Определим обучающий и валидационный датасеты
train_dataset = CaltechDataset(df_train, transforms_train)
val_dataset = CaltechDataset(df_val, transforms_val)
image_datasets = {
        "train": train_dataset,
        "val": val_dataset,
    }

dataset_sizes = {
        data_type: len(image_datasets[data_type]) for data_type in ["train", "val"]
    }

# 5. Определим даталоадеры
dataloaders = {
        data_type: DataLoader(
            image_datasets[data_type],
            batch_size=32,
            shuffle=True
        )
        for data_type in ["train", "val"]
    }

Обучение модели

In [ ]:
if loss == "crossentropy":
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
elif loss == "crossentropybalance":
    # получим веса для каждого класса
    all_train_lbls = []
    for item, lbl in tqdm(train_dataset):
        all_train_lbls.append(lbl)
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.array(all_train_lbls)), y=np.array(all_train_lbls))
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights, reduction='mean', label_smoothing=label_smoothing)
elif loss == "kldiv":
    criterion = nn.KLDivLoss()

    
res_dict = train_model(
    model,
    criterion,
    optimizer,
    scheduler,
    dataloaders,
    dataset_sizes,
    num_epochs,
    save_path=f"./"
        )

In [ ]:
res_df = pd.DataFrame({
    "train_lst": res_dict["train_lst"],
    "val_lst": res_dict["val_lst"],
    "val_acc_lst": res_dict["val_acc_lst"],
    "lr_lst": res_dict["lr_lst"]
})

res_df.to_csv("res_df.csv")